Importing Libraries

In [31]:
#libraries needed
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
## Download Once #run once and comment it out to avoid it downloading multiple times
#nltk.download('stopwords') 
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 

from wordcloud import ImageColorGenerator
from PIL import Image

import warnings
%matplotlib inline

Fetchning new tweets in CSV Format

In [5]:
# Run this code only once. If sentiments.csv exists in the working directly , no need to run this again.

# query = "(democrats OR republican OR vote OR #election2022 OR midterms OR #midterm2022 OR #vote2022 OR #democrats OR #republicans OR #GOP ) since:2022-07-01 until:2022-11-02"
# tweets = []
# for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
#     if i>20000:
#         break
#     else:
#         tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
# df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
# df.to_csv('sentiment.csv', mode = 'a')

In [11]:
df =  pd.read_csv('sentiment.csv', encoding = 'unicode_escape')
df.head(2)

,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,0,2022-11-01 23:59:59+00:00,1587595340760457218,https://twitter.com/loremipsum_2020/status/158...,loremipsum_2020,Twitter for iPhone,NaN,@GHOSTofSURF @ThatEricAlper Always on spot! ð¤ª,0,0
1,1,2022-11-01 23:59:59+00:00,1587595340458463233,https://twitter.com/grtamericanovel/status/158...,grtamericanovel,Twitter Web App,"Connecticut, USA",@GreenwaldEd @MadeInTheUSANJ @RepMTG If #GOP g...,2,1


In [12]:
df.location.isna().sum()

8155

In [13]:
df['location'] = df['location'].fillna('Unknown')

Data Processing


In [17]:
# download stopwords locally - check readme.
eng_stop_words = list(stopwords.words('english'))

In [25]:
emoji = list(UNICODE_EMOJI.keys())

In [33]:
# function for preprocessing tweet in preparation for sentiment analysis
def ProcessedTweets(text):
    #changing tweet text to small letters
    text = text.lower()
    # Removing @ and links 
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
    # removing repeating characters
    text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
    # removing punctuation and numbers
    punct = str.maketrans('', '', string.punctuation+string.digits)
    text = text.translate(punct)
    # tokenizing words and removing stop words from the tweet text
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    filtered_words = [w for w in filtered_words if w not in emoji]
    # lemmetizing words
    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)
    return text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...


True

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...


True

In [39]:
# Generate a new column called 'Processed Tweets' by applying preprocessed tweets function to the 'Tweet' column.
df['Processed_Tweets'] = df['tweet'].apply(ProcessedTweets)

In [40]:
df.head(5)

,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet,Processed_Tweets
0,0,2022-11-01 23:59:59+00:00,1587595340760457218,https://twitter.com/loremipsum_2020/status/158...,loremipsum_2020,Twitter for iPhone,Unknown,@GHOSTofSURF @ThatEricAlper Always on spot! ð¤ª,0,0,always spot ð¤ª
1,1,2022-11-01 23:59:59+00:00,1587595340458463233,https://twitter.com/grtamericanovel/status/158...,grtamericanovel,Twitter Web App,"Connecticut, USA",@GreenwaldEd @MadeInTheUSANJ @RepMTG If #GOP g...,2,1,get majority amp dont mtg cld head house cmte ...
2,2,2022-11-01 23:59:59+00:00,1587595339992731649,https://twitter.com/buildurlife/status/1587595...,buildurlife,Twitter for Android,BBB ð«§,Let's vote! We have a chance to give them more...,0,0,let vote chance give exposure social medium do...
3,3,2022-11-01 23:59:59+00:00,1587595339783360513,https://twitter.com/MMayhem2020/status/1587595...,MMayhem2020,Twitter for iPhone,Red State USA,This is @KathyHochul NY. Get out and vote for ...,0,0,ny get vote
4,4,2022-11-01 23:59:59+00:00,1587595339653173258,https://twitter.com/trevormdale/status/1587595...,trevormdale,Twitter for Android,"Minnesota, USA",@tophat595 @KihneSheila Thanks democrats! You ...,1,0,thanks democrat ruin everything


Sentiment analysis


In [41]:
# Function for polarity score
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

# Function to get sentiment type
#setting the conditions
def sentimenttextblob(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive" 

In [42]:
# using the functions to get the polarity and sentiment
df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df['Sentiment'] = df['Polarity'].apply(sentimenttextblob)
sent = df['Sentiment'].value_counts()
sent

Neutral     10823
Positive     6181
Negative     2997
Name: Sentiment, dtype: int64